In [48]:
"""
2025-09-02 MY

PyTorch-TCN: https://github.com/paul-krug/pytorch-tcn/tree/main

#pip install --user pytorch-tcn


"""

import numpy as np
from pytorch_tcn import TCN
import pandas as pd
#import xarray as xr
import matplotlib.pyplot as plt
#import re

In [26]:
fp = '/scratch/project_2009889/UN-EO-data/finland-data-v2/data/2024-06-15_2024-06-30_p1d/metrics-numpy/samples-wide-complete.columns.csv'
samples_wide_complete_columns = pd.read_csv(fp).columns.tolist()


fp = '/scratch/project_2009889/UN-EO-data/finland-data-v2/data/2024-06-15_2024-06-30_p1d/metrics-numpy/samples-wide-complete.npz'
samples_wide_complete = np.load(fp)

In [27]:
# take only bin variables:
chosen_idx = [n for n, x in enumerate(samples_wide_complete_columns) if 'bin' in x]

# take statistical variables:
#chosen_idx2 = [i for i, item in enumerate(samples_wide_complete_columns) if item.endswith(('sd', 'mean', 'median'))]

In [34]:
# this is needed to get the labels and dates rights:
df = pd.read_parquet('/scratch/project_2009889/UN-EO-data/finland-data-v2/data/2024-06-15_2024-06-30_p1d/metrics-parquet/samples-wide-complete.parquet')

# prepare y:
# preserves the order of labels:
itemID = df['label'].drop_duplicates() 

# read y:
dfy = pd.read_csv('/scratch/project_2009889/UN-EO-data/wheatFields2024_y.csv')

#reorder y:
dfyreordered = dfy.set_index('parcelID').reindex(itemID)#.reset_index()

In [24]:
databins = samples_wide_complete[:, chosen_idx[0]:]
# reshape into 3D (observations, times, features):
nrobs = len(set(df['label']))
times = len(df['Index'].unique())
data3Dbins = np.reshape(databins, (nrobs, times, -1))
data3Dbins.shape

#datastat = samples_wide_complete[:, chosen_idx2[0]:chosen_idx2[-1]]
#nrobs = len(set(df['label']))
#times = len(df['Index'].unique())
#data3Dstat = np.reshape(datastat, (nrobs, times, -1))
#data3Dstat.shape

data3Dbins.shape#, data3Dstat.shape

(402, 14, 160)

In [14]:
# Normalization of bin data:
def _l1_normalization(a):
    return (np.nan_to_num(a/a.sum()))

nrbins = 32
nrbands = int(data3Dbins.shape[2]/nrbins)

xtrain4d = data3Dbins.reshape(data3Dbins.shape[0], data3Dbins.shape[1], nrbands, -1)  
xtrain4d.shape # (observations, times, features, bins)

(402, 14, 5, 32)

In [16]:
# data normalized along the bin axis:
normalized = np.apply_along_axis(_l1_normalization, 3, xtrain4d)

In [18]:
normalized.shape

(402, 14, 5, 32)

In [19]:
# back to 3D:
m,n = normalized.shape[:2]
normalized3D = normalized.reshape(m,n,-1) 
normalized3D.shape

(402, 14, 160)

In [39]:
y = dfyreordered['y'].values
y.shape

(402,)

In [43]:
# Run TCN:

# in Keras:
#model = TCN(xtrain3d.shape[1], xtrain3d.shape[2], nbfilters, kernelsize, dilations)

# in PyTorch:
model = TCN(num_inputs = normalized3D.shape[2], num_channels = [32, 32, 32, 32], kernel_size = 2, dilations = [1, 2, 4, 8], input_shape = 'NLC', causal = True)

In [46]:
print(model)

TCN(
  (network): ModuleList(
    (0): TemporalBlock(
      (conv1): ParametrizedTemporalConv1d(
        160, 32, kernel_size=(2,), stride=(1,)
        (padder): TemporalPad1d(
          (pad): ConstantPad1d(padding=(1, 0), value=0.0)
        )
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): _WeightNorm()
          )
        )
      )
      (conv2): ParametrizedTemporalConv1d(
        32, 32, kernel_size=(2,), stride=(1,)
        (padder): TemporalPad1d(
          (pad): ConstantPad1d(padding=(1, 0), value=0.0)
        )
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): _WeightNorm()
          )
        )
      )
      (activation1): ReLU()
      (activation2): ReLU()
      (activation_final): ReLU()
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (downsample): Conv1d(160, 32, kernel_size=(1,), stride=(1,))
    )
    (1): TemporalBlock(